<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8%20Python%20%D0%B4%D0%BB%D1%8F%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0/%D0%94%D0%97_%D0%9A%D0%BE%D0%BD%D0%B4%D1%80%D0%B0%D1%82%D1%8C%D0%B5%D0%B2_NUMPY-48%20%D0%BA%20%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'

In [2]:
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.text)
soup

In [ ]:
news = soup.find_all('article', class_='tm-articles-list__item')
print(len(news))
print(news[0])

In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
soup = BeautifulSoup(res.text)
news = soup.find_all('article', class_='tm-articles-list__item')

for article in news:
    # Проверка наличия заголовка в оригинальной разметке
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print("Заголовок:", title)
    else:
        # Проверка наличия заголовка в другой разметке
        title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
        if title_element_alt is not None:
            title = title_element_alt.find('span').text.strip()
            print("Заголовок:", title)
        else:
            print("Заголовок не найден")

    # Проверка наличия ссылки в оригинальной разметке
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        # Проверка наличия ссылки в другой разметке
        link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
        if link_element_alt is not None:
            relative_link = link_element_alt.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            print(full_link)
        else:
            print("Ссылка не найдена")

    # Проверка наличия даты в оригинальной разметке
    date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
    if date_element is not None:
        date = date_element.get('title')
        print("Дата:", date)
    else:
        # Проверка наличия даты в другой разметке
        date_element_alt = article.find('span', class_='tm-article-datetime-published').find('time')
        if date_element_alt is not None:
            date = date_element_alt.get('title')
            print("Дата:", date)
        else:
            print("Дата не найдена")

    print()


Заголовок: Пять лет Школе анализа данных
https://habr.com/ru/companies/yandex/articles/148443/
Дата: 2012-07-25, 13:29

Заголовок: 10 лет Школе анализа данных Яндекса
https://habr.com/ru/companies/yandex/articles/334066/
Дата: 2017-07-25, 13:47

Заголовок: Как за две недели проверить гипотезы применимости анализа данных в горно-металлургической компании
https://habr.com/ru/companies/factory5/articles/681846/
Дата: 2022-08-10, 15:52

Заголовок: Что такое бессерверный SQL? И как использовать его для анализа данных?
https://habr.com/ru/companies/microsoft/articles/537064/
Дата: 2021-01-14, 10:00

Заголовок: Как мы строим систему обработки, хранения и анализа данных в СИБУРе
https://habr.com/ru/companies/sibur_official/articles/436632/
Дата: 2019-01-21, 11:21

Заголовок: Становясь Пангеей: будущее современного стека для анализа данных
https://habr.com/ru/articles/763942/
Дата: 2023-09-27, 19:55

Заголовок: В VK Cloud стал доступен инструмент поиска и анализа данных OpenSearch
https://habr.

In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
soup = BeautifulSoup(res.text)
news = soup.find_all('article', class_='tm-articles-list__item')

for article in news:
    # Проверка наличия заголовка в оригинальной разметке
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print("Заголовок:", title)
    else:
        # Проверка наличия заголовка в другой разметке
        title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
        if title_element_alt is not None:
            title = title_element_alt.find('span').text.strip()
            print("Заголовок:", title)
        else:
            print("Заголовок не найден")

    # Проверка наличия ссылки в оригинальной разметке
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        # Проверка наличия ссылки в другой разметке
        link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
        if link_element_alt is not None:
            relative_link = link_element_alt.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            print(full_link)
        else:
            print("Ссылка не найдена")

Заголовок: Пять лет Школе анализа данных
https://habr.com/ru/companies/yandex/articles/148443/
Заголовок: 10 лет Школе анализа данных Яндекса
https://habr.com/ru/companies/yandex/articles/334066/
Заголовок: Как за две недели проверить гипотезы применимости анализа данных в горно-металлургической компании
https://habr.com/ru/companies/factory5/articles/681846/
Заголовок: Что такое бессерверный SQL? И как использовать его для анализа данных?
https://habr.com/ru/companies/microsoft/articles/537064/
Заголовок: Как мы строим систему обработки, хранения и анализа данных в СИБУРе
https://habr.com/ru/companies/sibur_official/articles/436632/
Заголовок: Становясь Пангеей: будущее современного стека для анализа данных
https://habr.com/ru/articles/763942/
Заголовок: В VK Cloud стал доступен инструмент поиска и анализа данных OpenSearch
https://habr.com/ru/news/740666/
Заголовок: Ubic создаст платформу для анализа данных о москвичах
https://habr.com/ru/news/501944/
Заголовок: Microsoft анонсировал

In [23]:
for article in news:
    # Проверка наличия заголовка в оригинальной разметке
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print("Заголовок:", title)
    else:
        # Проверка наличия заголовка в другой разметке
        title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
        if title_element_alt is not None:
            title = title_element_alt.find('span').text.strip()
            print("Заголовок:", title)
        else:
            print("Заголовок не найден")

    # Проверка наличия ссылки
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        print("Ссылка не найдена")

    print()

Заголовок: Пять лет Школе анализа данных
Ссылка не найдена

Заголовок: 10 лет Школе анализа данных Яндекса
Ссылка не найдена

Заголовок: Как за две недели проверить гипотезы применимости анализа данных в горно-металлургической компании
Ссылка не найдена

Заголовок: Что такое бессерверный SQL? И как использовать его для анализа данных?
Ссылка не найдена

Заголовок: Как мы строим систему обработки, хранения и анализа данных в СИБУРе
Ссылка не найдена

Заголовок: Становясь Пангеей: будущее современного стека для анализа данных
Ссылка не найдена

Заголовок: В VK Cloud стал доступен инструмент поиска и анализа данных OpenSearch
Ссылка не найдена

Заголовок: Ubic создаст платформу для анализа данных о москвичах
Ссылка не найдена

Заголовок: Microsoft анонсировала запуск «Планетарного компьютера» для сбора, хранения и анализа данных о состоянии Земли
Ссылка не найдена

Заголовок: Владимир Путин выступил с Германом Грефом на конференции по ИИ и анализу данных Artificial Intelligence Journey 20

In [21]:
for article in news:
    # Проверка наличия заголовка
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print(title)
    else:
        print("Заголовок не найден")
    # Проверка наличия ссылки
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        print("Ссылка не найдена")

Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Новый подход к спортивному анализу данных: какие шаблоны «рвет» SDSJ
https://habr.com/ru/specials/344806/
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена
Заголовок не найден
Ссылка не найдена


In [9]:
for article in news:
    title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком
    if title_element is None:
        title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
    if title_element is not None:
        title = title_element.text.strip()  # получаем текст заголовка
        print(title)
        relative_link = title_element.get('href')  # получаем относительную ссылку
        full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
        print(full_link)
        date_element = article.find('span', class_='tm-article-datetime-published').find('time')
        if date_element is None:
            date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
        if date_element is not None:
            date = date_element.get('title')
            print(date)
            print()

Пять лет Школе анализа данных
https://habr.com/ru/companies/yandex/articles/148443/
2012-07-25, 13:29

10 лет Школе анализа данных Яндекса
https://habr.com/ru/companies/yandex/articles/334066/
2017-07-25, 13:47

Как за две недели проверить гипотезы применимости анализа данных в горно-металлургической компании
https://habr.com/ru/companies/factory5/articles/681846/
2022-08-10, 15:52

Что такое бессерверный SQL? И как использовать его для анализа данных?
https://habr.com/ru/companies/microsoft/articles/537064/
2021-01-14, 10:00

Как мы строим систему обработки, хранения и анализа данных в СИБУРе
https://habr.com/ru/companies/sibur_official/articles/436632/
2019-01-21, 11:21

Становясь Пангеей: будущее современного стека для анализа данных
https://habr.com/ru/articles/763942/
2023-09-27, 19:55

В VK Cloud стал доступен инструмент поиска и анализа данных OpenSearch
https://habr.com/ru/news/740666/
2023-06-08, 13:18

Ubic создаст платформу для анализа данных о москвичах
https://habr.com/ru/

AttributeError: ignored

In [ ]:
def get_habr_posts(query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title_element = article.find('a', 'tm-title__link')
            title = title_element.text
            relative_link = title_element.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            date_element = article.find('span', class_='tm-article-datetime-published').find('time')
            date = date_element.get('title')
            row = {'date': date, 'title': title, 'link': full_link}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['анализ данных'])
res

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

def get_habr_posts(query):
    base_url = 'https://habr.com'
    habr_blog = pd.DataFrame()
    for q in query:
        URL = f'{base_url}/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком
            if title_element is None:
                title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
            if title_element is not None:
                title = title_element.text.strip()  # получаем текст заголовка
                print(title)
                relative_link = title_element.get('href')  # получаем относительную ссылку
                full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
                print(full_link)
                date_element = article.find('span', class_='tm-article-datetime-published').find('time')
                if date_element is None:
                    date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
                if date_element is not None:
                    date = date_element.get('title')
                    print(date)
                row = {'date': date, 'title': title, 'link': full_link}
                habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
print(res)


Как устроен GIL (Global Interpreter Lock) в Python: влияние на многозадачность и производительность
https://habr.com/ru/companies/otus/articles/769448/
2023-10-24, 13:48
Python-разработчик: из новичка в профессионалы. Как начать свой путь в освоении языка?
https://habr.com/ru/companies/productstar/articles/769134/
2023-10-22, 19:07
Go vs Python. Виталий Левченко
https://habr.com/ru/articles/488644/
2020-02-17, 10:34
Что выбрать новичку: Python или Java?
https://habr.com/ru/companies/productstar/articles/769680/
2023-10-25, 13:34
Python Lives in Excel
https://habr.com/ru/articles/770006/
2023-10-26, 08:42
У нас в Excel поселился замечательный Python
https://habr.com/ru/articles/769732/
2023-10-25, 08:26
Искусственный интеллект на Python с использованием TensorFlow и Keras
https://habr.com/ru/articles/770554/
2023-10-29, 12:40
Python декораторы на максималках. Универсальный рецепт по написанию и аннотированию от мала до велика
https://habr.com/ru/articles/750312/
2023-07-25, 20:02
Решени

AttributeError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

def get_habr_posts(query):
    base_url = 'https://habr.com'
    habr_blog = pd.DataFrame()

    for q in query:
        URL = f'{base_url}/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        articles = soup.find_all('article', class_='tm-articles-list__item')

        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')  # поиск элемента с заголовком

            if title_element is None:
                title_element = article.find('a', class_='tm-title__link')  # если не найден, ищем элемент с ссылкой
                title = title_element.text.strip()  # получаем текст заголовка
                relative_link = title_element.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)  # создаем полную ссылку
                date_element = article.find('span', class_='tm-article-datetime-published').find('time')
                date = date_element.get('title')

            if title is not None:
                title = title_element.text.strip()  # получаем текст заголовка
                relative_link = article.find('a', 'full_link')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date = article.find('time', class_='tm-megapost-snippet__datetime-published')

        row = {'date': date, 'title': title, 'link': full_link}
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])

    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
res

,date,title,link
0,None,Три уровня погружения в Python. Запись докладо...,https://habr.com
1,None,Обзор наиболее интересных материалов по анализ...,https://habr.com


Код выполняет следующие действия:

1. Создание функции `get_habr_posts`, которая принимает в качестве параметра список `query`. В этой функции будет выполнен поиск статей на сайте habr.com по каждому ключевому слову из списка `query`.

2. Создание пустого DataFrame с именем `habr_blog`, в котором будут храниться данные о статьях.

3. В цикле `for q in query` происходит перебор каждого ключевого слова в списке `query`. Для каждого ключевого слова инициализируется GET-запрос к URL-адресу "https://habr.com/ru/search/" со значением параметра `q` в качестве поискового запроса.

4. Затем выполняется парсинг HTML-страницы с помощью BeautifulSoup, чтобы найти все статьи на странице. При помощи метода `find_all` ищутся все элементы `<article>` с классом `"tm-articles-list__item"`.

5. Внутри вложенного цикла `for article in articles` происходит обработка каждого элемента `<article>`, чтобы извлечь информацию о дате публикации, названии статьи и ссылке на нее.

6. Извлеченные данные сохраняются в словаре `row` с ключами `'date'`, `'title'` и `'link'`.

7. С помощью конкатенации DataFrame (`pd.concat([habr_blog, pd.DataFrame([row])])`) данные из словаря `row` добавляются как новая строка в DataFrame `habr_blog`.

8. В конце функция возвращает DataFrame `habr_blog`, содержащий информацию о статьях.

9. После вызова функции `get_habr_posts(['python', 'data science'])` результат сохраняется в переменной `res`.

10. Для удаления дубликатов в DataFrame `res` применяется метод `drop_duplicates()`. Этот метод удаляет полностью дублирующие строки из DataFrame.

11. Затем результаты выводятся на экран путем вызова `res`.

## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
